In [19]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
import tensorflow.keras as keras
from tensorflow.keras.optimizers import Adam

In [20]:
if tf.config.list_physical_devices('GPU'):
  print("GPU is available")
  print("CUDA Available: ", tf.config.list_physical_devices('GPU'))
  
else:
  print("GPU is not available")

GPU is available
CUDA Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [21]:
cwd = os.getcwd()
timeseries_df = pd.read_csv(cwd + "/datasets/timeseries_with_wav.csv")
timeseries_df = timeseries_df.drop('Unnamed: 0', axis=1)
timeseries_df = timeseries_df.drop(columns=['datetime'])

result_df = pd.read_csv(cwd + "/datasets/patient.csv")
result_df = result_df.drop('Unnamed: 0', axis=1)
filtered_df = result_df[(result_df['result'] == 'LA') | (result_df['result'] == 'LA DİLATASYONU')]
filtered_df.loc[:, 'result'] = filtered_df['result'].apply(lambda x: 0 if x == 'LA' else 1)
timeseries_df = pd.merge(timeseries_df, filtered_df, on='patient')

personal_df = pd.read_csv(cwd + "/datasets/patient_info.csv")
personal_df = personal_df.drop('Unnamed: 0', axis=1)

mean_weight = personal_df[personal_df.sex == 'K']['weight'].mean()
mean_height = personal_df[personal_df.sex == 'K']['height'].mean()

personal_df.loc[personal_df.id == 4, 'height'] = mean_height
personal_df.loc[personal_df.id == 30, 'weight'] = mean_weight
personal_df.loc[personal_df.id == 30, 'height'] = mean_height

mean_sys_w = personal_df[personal_df.sex == 'K']['systolic_pressure'].mean()
mean_dia_W = personal_df[personal_df.sex == 'K']['diastolic_pressure'].mean()

mean_sys_m = personal_df[personal_df.sex == 'E']['systolic_pressure'].mean()
mean_dia_m = personal_df[personal_df.sex == 'E']['diastolic_pressure'].mean()


personal_df.loc[personal_df.id == 33, 'systolic_pressure'] = mean_sys_w
personal_df.loc[personal_df.id == 33, 'diastolic_pressure'] = mean_dia_W

personal_df.loc[personal_df.id == 36, 'systolic_pressure'] = mean_sys_m
personal_df.loc[personal_df.id == 36, 'diastolic_pressure'] = mean_dia_m

df = pd.merge(timeseries_df, personal_df.drop(columns=['name_surname', 'date', 'NOT', 'complaint', 'identification_number', 'systolic_pressure', 'diastolic_pressure']), left_on='patient', right_on='id')
df = df.drop('id', axis=1)

print(df.shape)
df.head()

(547277, 20)


,Millis,ECG,IR,RED,patient,rate_avg,rate_std,rate_min,rate_max,result,sex,HT,DM,HL,KAH,KOAH,KBY,KC HST,height,weight
0,892,226.0,28556.0,28966.0,1,0.0,0.0,0,0,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,158.0,95.0
1,893,226.0,28554.0,28959.0,1,0.0,0.0,0,0,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,158.0,95.0
2,893,226.0,28556.0,28960.0,1,0.0,0.0,0,0,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,158.0,95.0
3,893,226.0,28552.0,28959.0,1,0.0,0.0,0,0,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,158.0,95.0
4,894,226.0,28551.0,28954.0,1,0.0,0.0,0,0,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,158.0,95.0


In [22]:
# the patient has not any info for other diseases
df = df.dropna()
df.result = df.result.astype('int')
df.sex = df.sex.astype('category')
df.sex = df.sex.cat.codes

norm_cols = ['ECG', 'IR', 'RED', 'rate_avg', 'rate_std', 'rate_min', 'rate_max', 'height', 'weight']
scaler = StandardScaler()
df[norm_cols] = scaler.fit_transform(df[norm_cols])
n_classes = len(np.unique(df.result))

In [23]:
y_true_list = []
y_pred_list = []
y_pred_bool_list = []
model_list = []

In [24]:
timesteps = 4096
features = 19

In [8]:
# Assuming df is your DataFrame and build_model is your model building function
patients = df['patient'].unique()
np.random.shuffle(patients)
kf = KFold(n_splits=len(patients) // 8)

for train_index, test_index in kf.split(patients):
    train_patients = patients[train_index]
    test_patients = patients[test_index]

    train_data = df[df['patient'].isin(train_patients)]
    test_data = df[df['patient'].isin(test_patients)]

    train_X = train_data.drop('result', axis=1)
    train_y = train_data.result
    test_X = test_data.drop('result', axis=1)
    test_y = test_data.result

    # Ensure train_y and test_y are binary (0 or 1)
    train_y = train_y.astype(np.float32)
    test_y = test_y.astype(np.float32)
    
    train_X = train_X.values.reshape((train_X.values.shape[0], train_X.values.shape[1], 1))
    test_X = test_X.values.reshape((test_X.values.shape[0], test_X.values.shape[1], 1))
    
    model = Sequential()
    model.add(GRU(250, input_shape=(timesteps, features), return_sequences=True))
    model.add(GRU(250))
    model.add(Dense(1, activation='sigmoid'))

    # Compiling the model
    learning_rate = 0.005
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    # Training the model
    model.fit(train_X, train_y, epochs=20, batch_size=4096)

    # Predicting on the same data (for simplicity)
    predictions = model.predict(test_X)
    preds = (predictions > 0.5).astype(int)

    # Evaluating the model
    loss, accuracy = model.evaluate(test_X, test_y)
    print(f'Loss: {loss}, Accuracy: {accuracy}')

    y_true_list.append(test_y)
    y_pred_list.append(predictions) 
    y_pred_bool_list.append(preds)
    model_list.append(model)

    print(classification_report(test_y, preds))

Epoch 1/20


/home/honorsea/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


110/110 ━━━━━━━━━━━━━━━━━━━━ 15s 116ms/step - accuracy: 0.8141 - loss: 0.3841
Epoch 2/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 13s 117ms/step - accuracy: 1.0000 - loss: 2.4122e-04
Epoch 3/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 13s 117ms/step - accuracy: 1.0000 - loss: 8.3844e-05
Epoch 4/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 13s 117ms/step - accuracy: 1.0000 - loss: 5.1807e-05
Epoch 5/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 13s 117ms/step - accuracy: 1.0000 - loss: 2.8208e-05
Epoch 6/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 13s 118ms/step - accuracy: 1.0000 - loss: 1.8493e-05
Epoch 7/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 13s 118ms/step - accuracy: 1.0000 - loss: 1.4399e-05
Epoch 8/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 13s 118ms/step - accuracy: 1.0000 - loss: 1.0513e-05
Epoch 9/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 13s 118ms/step - accuracy: 1.0000 - loss: 8.6427e-06
Epoch 10/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 16s 141ms/step - accuracy: 1.0000 - loss: 6.9300e-06
Epoch 11/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 13s 119ms/step - accuracy: 1.0000 - loss: 5

/home/honorsea/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


100/100 ━━━━━━━━━━━━━━━━━━━━ 13s 118ms/step - accuracy: 0.7278 - loss: 0.4828
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 0.9996 - loss: 0.0019
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 1.0000 - loss: 3.1720e-04
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 1.0000 - loss: 1.4878e-04
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 1.0000 - loss: 7.8536e-05
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 119ms/step - accuracy: 1.0000 - loss: 6.9879e-05
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 119ms/step - accuracy: 1.0000 - loss: 4.7571e-05
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 119ms/step - accuracy: 1.0000 - loss: 2.8520e-05
Epoch 9/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 119ms/step - accuracy: 0.9734 - loss: 0.0828
Epoch 10/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 119ms/step - accuracy: 1.0000 - loss: 2.1665e-04
Epoch 11/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 119ms/step - accuracy: 1.0000 - loss: 1.0695e-0

...

106/106 ━━━━━━━━━━━━━━━━━━━━ 14s 117ms/step - accuracy: 0.7929 - loss: 0.3980
Epoch 2/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 13s 118ms/step - accuracy: 0.9989 - loss: 0.0050
Epoch 3/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 13s 119ms/step - accuracy: 0.9999 - loss: 7.3471e-04
Epoch 4/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 13s 119ms/step - accuracy: 1.0000 - loss: 2.4407e-04
Epoch 5/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 13s 119ms/step - accuracy: 1.0000 - loss: 1.2614e-04
Epoch 6/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 13s 119ms/step - accuracy: 1.0000 - loss: 7.2907e-05
Epoch 7/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 13s 119ms/step - accuracy: 1.0000 - loss: 4.9993e-05
Epoch 8/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 13s 119ms/step - accuracy: 1.0000 - loss: 3.5167e-05
Epoch 9/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 13s 119ms/step - accuracy: 1.0000 - loss: 2.8023e-05
Epoch 10/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 13s 124ms/step - accuracy: 1.0000 - loss: 2.1012e-05
Epoch 11/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 13s 120ms/step - accuracy: 1.0000 - loss: 1.717

/home/honorsea/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


98/98 ━━━━━━━━━━━━━━━━━━━━ 13s 118ms/step - accuracy: 0.8206 - loss: 0.3643
Epoch 2/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 1.0000 - loss: 3.3190e-04
Epoch 3/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 1.0000 - loss: 7.9464e-05
Epoch 4/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 1.0000 - loss: 3.6784e-05
Epoch 5/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 12s 119ms/step - accuracy: 1.0000 - loss: 2.2359e-05
Epoch 6/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 12s 119ms/step - accuracy: 1.0000 - loss: 1.5745e-05
Epoch 7/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 1.0000 - loss: 1.1870e-05
Epoch 8/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 1.0000 - loss: 9.0832e-06
Epoch 9/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 1.0000 - loss: 7.3298e-06
Epoch 10/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 1.0000 - loss: 6.0041e-06
Epoch 11/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 1.0000 - loss: 4.8747e-06
Epoch 12/20


/home/honorsea/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 117ms/step - accuracy: 0.7860 - loss: 0.4300
Epoch 2/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 13s 118ms/step - accuracy: 0.9996 - loss: 0.0066
Epoch 3/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 13s 118ms/step - accuracy: 1.0000 - loss: 4.5394e-04
Epoch 4/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 13s 118ms/step - accuracy: 1.0000 - loss: 1.9489e-04
Epoch 5/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 13s 118ms/step - accuracy: 1.0000 - loss: 1.1564e-04
Epoch 6/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 13s 119ms/step - accuracy: 1.0000 - loss: 8.5665e-05
Epoch 7/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 13s 118ms/step - accuracy: 1.0000 - loss: 4.8451e-05
Epoch 8/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 13s 118ms/step - accuracy: 1.0000 - loss: 3.4183e-05
Epoch 9/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 13s 119ms/step - accuracy: 1.0000 - loss: 2.6774e-05
Epoch 10/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 13s 118ms/step - accuracy: 1.0000 - loss: 2.1485e-05
Epoch 11/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 15s 141ms/step - accuracy: 1.0000 - loss: 1.734

In [10]:
for i in range(5):
    print("------------------", (i+1), "------------------")
    print(classification_report(y_true_list[i], y_pred_bool_list[i]))

------------------ 1 ------------------
              precision    recall  f1-score   support

         0.0       0.43      0.96      0.59     26865
         1.0       0.96      0.37      0.54     55144

    accuracy                           0.57     82009
   macro avg       0.69      0.67      0.56     82009
weighted avg       0.78      0.57      0.55     82009

------------------ 2 ------------------
              precision    recall  f1-score   support

         0.0       1.00      0.24      0.39     48965
         1.0       0.67      1.00      0.80     74003

    accuracy                           0.70    122968
   macro avg       0.83      0.62      0.60    122968
weighted avg       0.80      0.70      0.64    122968

------------------ 3 ------------------
              precision    recall  f1-score   support

         0.0       0.67      0.69      0.68     35054
         1.0       0.82      0.81      0.82     62870

    accuracy                           0.77     97924
   macro

In [25]:
timesteps = 2048
features = 12

In [26]:
y_true_list1 = []
y_pred_list1 = []
y_pred_bool_list1 = []
model_list1 = []

In [27]:
df.columns

Index(['Millis', 'ECG', 'IR', 'RED', 'patient', 'rate_avg', 'rate_std',
       'rate_min', 'rate_max', 'result', 'sex', 'HT', 'DM', 'HL', 'KAH',
       'KOAH', 'KBY', 'KC HST', 'height', 'weight'],
      dtype='object')

In [28]:
df = df[['Millis', 'ECG', 'IR', 'RED', 'patient', 'rate_avg', 'rate_std',
       'rate_min', 'rate_max', 'result', 'sex', 'height', 'weight']]

In [29]:
# Assuming df is your DataFrame and build_model is your model building function
patients = df['patient'].unique()
np.random.shuffle(patients)
kf = KFold(n_splits=len(patients) // 8)

for train_index, test_index in kf.split(patients):
    train_patients = patients[train_index]
    test_patients = patients[test_index]

     

/home/honorsea/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 17s 28ms/step - accuracy: 0.9005 - loss: 0.1981
Epoch 2/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 1.0292e-04
Epoch 3/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 1.0000 - loss: 1.2752e-05
Epoch 4/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 12s 27ms/step - accuracy: 1.0000 - loss: 5.3456e-06
Epoch 5/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 12s 27ms/step - accuracy: 1.0000 - loss: 3.2453e-06
Epoch 6/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 2.1076e-06
Epoch 7/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 1.4867e-06
Epoch 8/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 1.0650e-06
Epoch 9/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 7.9711e-07
Epoch 10/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 6.0669e-07
Epoch 11/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 4

/home/honorsea/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
420/420 ━━━━━━━━━━━━━━━━━━━━ 14s 28ms/step - accuracy: 0.9208 - loss: 0.1638
Epoch 2/20
420/420 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 2.0997e-04
Epoch 3/20
420/420 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 6.8356e-05
Epoch 4/20
420/420 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 1.6269e-05
Epoch 5/20
420/420 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 4.9377e-06
Epoch 6/20
420/420 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 3.1602e-06
Epoch 7/20
420/420 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 2.1480e-06
Epoch 8/20
420/420 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 1.5673e-06
Epoch 9/20
420/420 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 1.1376e-06
Epoch 10/20
420/420 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 8.5094e-07
Epoch 11/20
420/420 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 6

/home/honorsea/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
389/389 ━━━━━━━━━━━━━━━━━━━━ 44s 101ms/step - accuracy: 0.8682 - loss: 0.2822
Epoch 2/20
389/389 ━━━━━━━━━━━━━━━━━━━━ 39s 100ms/step - accuracy: 0.9996 - loss: 0.0018
Epoch 3/20
389/389 ━━━━━━━━━━━━━━━━━━━━ 39s 101ms/step - accuracy: 1.0000 - loss: 1.9572e-04
Epoch 4/20
389/389 ━━━━━━━━━━━━━━━━━━━━ 39s 100ms/step - accuracy: 1.0000 - loss: 7.2230e-05
Epoch 5/20
389/389 ━━━━━━━━━━━━━━━━━━━━ 39s 99ms/step - accuracy: 1.0000 - loss: 3.2856e-05
Epoch 6/20
389/389 ━━━━━━━━━━━━━━━━━━━━ 39s 100ms/step - accuracy: 1.0000 - loss: 2.0684e-05
Epoch 7/20
389/389 ━━━━━━━━━━━━━━━━━━━━ 39s 100ms/step - accuracy: 1.0000 - loss: 1.3997e-05
Epoch 8/20
389/389 ━━━━━━━━━━━━━━━━━━━━ 39s 99ms/step - accuracy: 1.0000 - loss: 9.9825e-06
Epoch 9/20
389/389 ━━━━━━━━━━━━━━━━━━━━ 39s 100ms/step - accuracy: 1.0000 - loss: 7.4367e-06
Epoch 10/20
389/389 ━━━━━━━━━━━━━━━━━━━━ 39s 99ms/step - accuracy: 1.0000 - loss: 5.5288e-06
Epoch 11/20
389/389 ━━━━━━━━━━━━━━━━━━━━ 39s 99ms/step - accuracy: 1.0000 - loss

/home/honorsea/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
425/425 ━━━━━━━━━━━━━━━━━━━━ 59s 101ms/step - accuracy: 0.9146 - loss: 0.1833
Epoch 2/20
425/425 ━━━━━━━━━━━━━━━━━━━━ 43s 102ms/step - accuracy: 1.0000 - loss: 7.5633e-05
Epoch 3/20
425/425 ━━━━━━━━━━━━━━━━━━━━ 20s 47ms/step - accuracy: 0.9860 - loss: 0.0366
Epoch 4/20
425/425 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 0.9999 - loss: 0.0021
Epoch 5/20
425/425 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 3.8876e-04
Epoch 6/20
425/425 ━━━━━━━━━━━━━━━━━━━━ 20s 48ms/step - accuracy: 1.0000 - loss: 1.6691e-04
Epoch 7/20
425/425 ━━━━━━━━━━━━━━━━━━━━ 42s 99ms/step - accuracy: 1.0000 - loss: 9.2299e-05
Epoch 8/20
425/425 ━━━━━━━━━━━━━━━━━━━━ 41s 96ms/step - accuracy: 1.0000 - loss: 5.8216e-05
Epoch 9/20
425/425 ━━━━━━━━━━━━━━━━━━━━ 43s 101ms/step - accuracy: 0.9039 - loss: 0.1891
Epoch 10/20
425/425 ━━━━━━━━━━━━━━━━━━━━ 43s 101ms/step - accuracy: 0.9860 - loss: 0.0396
Epoch 11/20
425/425 ━━━━━━━━━━━━━━━━━━━━ 43s 102ms/step - accuracy: 1.0000 - loss: 0.0020
Epoch

/home/honorsea/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


418/418 ━━━━━━━━━━━━━━━━━━━━ 39s 88ms/step - accuracy: 0.9124 - loss: 0.1973
Epoch 2/20
418/418 ━━━━━━━━━━━━━━━━━━━━ 40s 96ms/step - accuracy: 0.9968 - loss: 0.0091
Epoch 3/20
418/418 ━━━━━━━━━━━━━━━━━━━━ 43s 102ms/step - accuracy: 0.9999 - loss: 7.7111e-04
Epoch 4/20
418/418 ━━━━━━━━━━━━━━━━━━━━ 42s 101ms/step - accuracy: 1.0000 - loss: 1.7874e-04
Epoch 5/20
418/418 ━━━━━━━━━━━━━━━━━━━━ 42s 101ms/step - accuracy: 1.0000 - loss: 6.1962e-05
Epoch 6/20
418/418 ━━━━━━━━━━━━━━━━━━━━ 42s 100ms/step - accuracy: 1.0000 - loss: 3.7255e-05
Epoch 7/20
418/418 ━━━━━━━━━━━━━━━━━━━━ 42s 101ms/step - accuracy: 1.0000 - loss: 2.5098e-05
Epoch 8/20
418/418 ━━━━━━━━━━━━━━━━━━━━ 42s 100ms/step - accuracy: 1.0000 - loss: 1.4916e-05
Epoch 9/20
418/418 ━━━━━━━━━━━━━━━━━━━━ 42s 99ms/step - accuracy: 1.0000 - loss: 9.9271e-06
Epoch 10/20
418/418 ━━━━━━━━━━━━━━━━━━━━ 42s 100ms/step - accuracy: 1.0000 - loss: 7.0796e-06
Epoch 11/20
418/418 ━━━━━━━━━━━━━━━━━━━━ 42s 100ms/step - accuracy: 1.0000 - loss: 5.0818e-

In [30]:
for i in range(5):
    print("------------------", (i+1), "------------------")
    print(classification_report(y_true_list1[i], y_pred_bool_list1[i]))

------------------ 1 ------------------
              precision    recall  f1-score   support

         0.0       0.38      0.42      0.40     40383
         1.0       0.57      0.53      0.55     58175

    accuracy                           0.48     98558
   macro avg       0.47      0.47      0.47     98558
weighted avg       0.49      0.48      0.49     98558

------------------ 2 ------------------
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00     29605
         1.0       0.68      0.86      0.76     71484

    accuracy                           0.61    101089
   macro avg       0.34      0.43      0.38    101089
weighted avg       0.48      0.61      0.54    101089

------------------ 3 ------------------
              precision    recall  f1-score   support

         0.0       0.58      1.00      0.74     34354
         1.0       1.00      0.75      0.86     98477

    accuracy                           0.81    132831
   macro